In [1]:
! pip install requests pandas gspread

In [2]:
from datetime import date, timedelta
import numpy as np
import pandas as pd
from data_project.tapdb import TapDBClient
from data_project.gsheets import DateSheet
from config import TAPDB_USER, TAPDB_PASS
import asyncio

target_date = (date.today() - timedelta(days=1)).isoformat()
#target_date = '2024-07-27'

if isinstance(target_date, str):
    target_date = date.fromisoformat(target_date)

async def create_tapdb():
    return TapDBClient(TAPDB_USER, TAPDB_PASS, target_date=target_date, tz_offset=1)

async def create_sheet():
    return DateSheet('[RP_DMM] Revenue Report', date=target_date.isoformat())

task_tapdb = asyncio.create_task(create_tapdb())
task_sheet = asyncio.create_task(create_sheet())

await task_tapdb
await task_sheet

tapdb = task_tapdb.result()
sheet = task_sheet.result()


In [3]:
# [RP_DMM] Revenue Report >> 活躍設備
sheet.change_sheet('Daily Re.')
data = tapdb.get_data('RP DMM (PRO)', 'active', dim='os', device=True)

sheet.update_cells('活躍設備\n(tap db)', int(data['DAU_DEVICE'].sum()))
sheet.update_cells('新活躍設備\n(tap db)', int(data['newDAU_DEVICE'].sum()))
data

,key,DAU_DEVICE,newDAU_DEVICE,DAU_DEVICE2,DAU_DEVICE3,DAU_DEVICE7,DAU_DEVICE14,DAU_DEVICE30
key,,,,,,,,
Android,Android,3722,99,3623,3576,3435,3270,2906


In [4]:
# [RP_DMM] Revenue Report >> 轉化率
data = tapdb.get_data('RP DMM (PRO)', 'source', dim='ver', pf='and')
row = data.loc[max(data.index)]

sheet.change_sheet('轉化率')
sheet.update_cells('版本', row['key'])
sheet.update_cells('新增設備', row['newDevices'])
sheet.update_cells('轉化設備', row['convertedDevices'])

data

,key,newPlayers,totalCharge,totalChargeAmount,firstChargePlayers,firstChargeAmount,newDevices,convertedDevices,devicesConvertRate,dayRetainedRate1,dayRetainedRate7,chargeRate,firstChargeRate,ARPU,ARPPU
key,,,,,,,,,,,,,,,
2.0.0.138,2.0.0.138,90,5,120.01,5,109.99,99,93,93.94,None,None,5.05,5.05,1.11,22
